In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import random

from tqdm import tqdm

In [23]:
RND_STATE = 777

Поскольку в дальнейшем планируется сравнение различных типов моделей, нам необходимо разделить наши данные на три выборки:
1. Тренировочная
2. Валидационная
3. Контрольная (тестовая)

Данное разделение необходимо для того, чтобы избежать переобучения на контрольную выборку.

In [24]:
data = pd.read_csv('Data\credit_train_processed.csv', encoding='cp1251', index_col=0)

In [25]:
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,59998.00,10,1_6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,10889.00,6,1_1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,10728.00,12,1_1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,12009.09,12,1_1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1_1,0.421385,SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


Разделим наши данные в соотношении 7:2:1.

In [26]:
X = data.drop('open_account_flg', axis=1)
y = data['open_account_flg']

X_train, X_1, y_train, y_1 = train_test_split(
    X, y, test_size=0.3, stratify=y, shuffle=True, random_state=RND_STATE)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_1, y_1, test_size=0.33, stratify=y_1, shuffle=True, random_state=RND_STATE)

In [27]:
train = X_train.join(y_train, how='left')
valid = X_valid.join(y_valid, how='left')
test = X_test.join(y_test, how='left')

In [28]:
datasets = [train, valid, test]
datasets_names = ["Тренировочный", "Валидационный", "Контрольный"]
for df in tqdm(datasets):
    print(f'{df.shape[0] / data.shape[0]:.2f}')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]

0.70
0.20
0.10


# Импутация пропусков

Давайте посмотрим количество пропусков по каждой переменной в наших наборах данных.

In [29]:
omission_info = pd.DataFrame(dict(zip(datasets_names, [df.isnull().sum() for df in datasets])))

In [30]:
omission_info

,Тренировочный,Валидационный,Контрольный
gender,0,0,0
age,2,0,1
marital_status,2,0,1
job_position,0,0,0
credit_sum,2,0,0
credit_month,0,0,0
tariff_id,0,0,0
score_shk,5,0,2
education,5,0,0
living_region,136,36,20


Пропуски в переменных будем проводить различными способами в зависимости от типа рассматриваемой переменной (категориальной или количественной). 

В случае *категориальной* переменной заполнение будем проводить <ins>модой</ins> **тренировочного** набора.

В случае *количественной* переменной заполнение проводится в зависимости от типа ее распределения (симметричный, ассиметричный):
* симметричный - можно воспользоваться <ins>средним</ins> значением **тренировочного** набора. Например, для параметра `age`.
* ассиметричный - лучше пользоваться <ins>медианой</ins> **тренировочного** набора, чтобы избежать влияния "черных лебедей".

Также можно заполнить значением выходящим за диапазон непропущенных значений, например, большим отрицательным числом.

## Количественные переменные

In [31]:
print(f'Количественные переменные: {test.dtypes[test.dtypes != object].index.values}')

Количественные переменные: ['age' 'credit_sum' 'credit_month' 'score_shk' 'monthly_income'
 'credit_count' 'overdue_credit_count' 'open_account_flg']


In [32]:
# Заполняем пропуски в переменных 'age', 'credit_sum', 'score_shk', 'monthly_income'
# медианой тренировочного набора.
for col in tqdm(['age', 'credit_sum', 'score_shk', 'monthly_income']):
    for df in datasets:
        df[col].fillna(train[col].median(), inplace=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 117.96it/s]


In [33]:
# Для переменных 'credit_count' и 'overdue_credit_count' заполним пропуски значением -1
for col in tqdm(['credit_count', 'overdue_credit_count']):
    for df in datasets:
        df[col].fillna(-1, inplace=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 668.47it/s]


## Категориальные переменные

In [34]:
print(f'Категориальные переменные: {test.dtypes[test.dtypes == object].index.values}')

Категориальные переменные: ['gender' 'marital_status' 'job_position' 'tariff_id' 'education'
 'living_region']


In [35]:
need_to_fix_categorical = omission_info.loc[test.dtypes[test.dtypes == object].index]\
                                        .sum(axis=1) > 0

In [36]:
need_to_fix_categorical[need_to_fix_categorical]

marital_status    True
education         True
living_region     True
dtype: bool

In [37]:
for cat in tqdm(['marital_status', 'education']):
    for df in datasets:
        df[cat].fillna(train[cat].mode().values[0], inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 32.34it/s]


In [38]:
for df in tqdm(datasets):
        df['living_region'].fillna('НЕ УКАЗАН', inplace=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 334.25it/s]


In [39]:
omission_info_after = pd.DataFrame(dict(zip(datasets_names, [df.isnull().sum() for df in datasets])))
omission_info_after

,Тренировочный,Валидационный,Контрольный
gender,0,0,0
age,0,0,0
marital_status,0,0,0
job_position,0,0,0
credit_sum,0,0,0
credit_month,0,0,0
tariff_id,0,0,0
score_shk,0,0,0
education,0,0,0
living_region,0,0,0


Далее сохраняем результаты нашей работы в файлы CSV, конструирование новых признаков будем проводить непосредственно перед построением соответственной модели.

# Конструирование новых признаков (Feature Engineering)

Начнем конструирование новых признаков, исходя из знаний предметной области.

In [40]:
print(f'Количественные переменные: {test.dtypes[test.dtypes != object].index}')

Количественные переменные: Index(['age', 'credit_sum', 'credit_month', 'score_shk', 'monthly_income',
       'credit_count', 'overdue_credit_count', 'open_account_flg'],
      dtype='object')


In [41]:
# Рассчитаем ежемесячный платеж по кредиту для всех датасетов
for df in tqdm(datasets):
    df['month_payment'] = df['credit_sum'] / df['credit_month']
# Заменим бесконечные значения, которые могли образоваться при делении, на -1
    df['month_payment'].replace([np.inf, -np.inf], -1, inplace=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 334.19it/s]


In [42]:
# Также важным параметром является какую долю зарплаты человек будет отдавать на погашение кредита
for df in tqdm(datasets):
    df['payment_income_part'] = df['month_payment'] / df['monthly_income']
# Заменим бесконечные значения, которые могли образоваться при делении, на -1    
    df['payment_income_part'].replace([np.inf, -np.inf], -1, inplace=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 273.46it/s]


In [43]:
datasets_en_names = ["Train_wo_na.csv", "Valid_wo_na.csv", "Control_wo_na.csv"]

for i, df in enumerate(datasets):
    df.to_csv(fr'Data\{datasets_en_names[i]}', encoding='cp1251')